In [1]:
import torch
import os
import json
from tqdm import tqdm
import numpy as np
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn as nn
import pickle
import gc
# Set this to disable warning messages in the generation mode.
transformers.utils.logging.set_verbosity_error()
import torch.nn.functional as F
import transformers.generation.logits_process as logits_process
import transformers.generation.stopping_criteria as stopping_criteria
from torch.nn import MSELoss,CrossEntropyLoss
from torch.cuda.amp import autocast, GradScaler
import time
from tqdm import trange

In [2]:
def LoadJson(path):
    '''
    '''
    res=[]
    with open(path,mode='r',encoding='utf-8') as f:
        dicts = json.load(f)
        res=dicts
    return res

def WriteJson(data,path):
    '''
    '''
    with open(path,'w',encoding='utf-8') as f:
        json.dump(data,f,indent=4)

In [3]:
CACHE_DIR ='/home/lenijwp/datacache/huggingface/'
device = 'cuda:1'
# model_name = 'TDC2023/trojan-large-pythia-6.9b'
model_path = '/home/lenijwp/codes/tdc2023-starter-kit/red_teaming/data/dev/base/model'
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16,low_cpu_mem_usage=True,device_map=device).eval()
use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side='left', use_fast=use_fast_tokenizer)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/lenijwp/.conda/envs/llmtest/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/lenijwp/.conda/envs/llmtest/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [4]:
hook_embeddings = []
# hook_outputs = []

def remove_hooks(model):
    for layer in model.children():
        if isinstance(layer, nn.Module):
            remove_hooks(layer)
        if hasattr(layer, '_forward_hooks'):
            layer._forward_hooks.clear()
        if hasattr(layer, '_backward_hooks'):
            layer._backward_hooks.clear()
remove_hooks(model)

def hook(module, fea_in, fea_out):
    hook_embeddings.append(fea_in)
    # hook_outputs.append(fea_out)
    return None

layer_name = 'lm_head'
for (name, module) in model.named_modules():
    if name == layer_name:
        print("")
        # module.requires_grad_(True)
        module.register_forward_hook(hook=hook)

def clear_hooks():
    global hook_embeddings
    # global hook_outputs
    hook_embeddings.clear()
    # hook_outputs.clear()
    torch.cuda.empty_cache()



In [5]:
def get_embeddings(text):
    global hook_embeddings
    
    input_ids = tokenizer(text, return_tensors="pt", truncation=True, padding=True).input_ids.to(device)
    model(input_ids)
    embeddings = hook_embeddings[-1][0][:,-1,:]
    ret = []
    for i in range(embeddings.shape[0]):
        ret.append(embeddings[i].cpu().detach().numpy())
    
    clear_hooks()
    
    return ret

In [7]:


Dataset = LoadJson('/home/lenijwp/codes/costello/data/initial_data/sst_tokens.json')

tmp = []

Embs=[]

for key in tqdm(list(Dataset.keys()),'Collecting'):
    tmp.append(str(key))

    if len(tmp)==20:
        response = get_embeddings(tmp)
        for ele in response:
            Embs.append(ele)
    
        tmp=[]
    



        
if len(tmp)!=0:
    response = get_embeddings(tmp)
    for ele in response:
        Embs.append(ele)
    time.sleep(1)
    tmp=[]


saveEmb = np.array(Embs)
print(saveEmb.shape)

np.save('./tokens.npy',saveEmb)


Dataset = LoadJson('/home/lenijwp/codes/costello/data/initial_data/sst_train.json')

tmp = []

Embs=[]

for key in tqdm(list(Dataset.keys()),'Collecting'):
    tmp.append(Dataset[key][0])

    if len(tmp)==10:
        response = get_embeddings(tmp)
        for ele in response:
            Embs.append(ele)
   
        tmp=[]
    
        
if len(tmp)!=0:
    response = get_embeddings(tmp)
    for ele in response:
        Embs.append(ele)
    time.sleep(1)
    tmp=[]

saveEmb = np.array(Embs)
print(saveEmb.shape)

np.save('./sst2-train.npy',saveEmb)


TestSuite = LoadJson('/home/lenijwp/codes/costello/data/contrast_set/ctset1.json')

for MuType in TestSuite.keys():

    # if str(MuType)=='synon_contr':
    #     continue
    print(MuType)
    #print(len(TestSuite[MuType]))

    Data = TestSuite[MuType]

    Embs=[]



    for i in trange(len(Data)):

        response = get_embeddings(Data[i])
        Embs.append([response[0], response[1], response[2]])

 

    saveEmb = np.array(Embs)
    print(saveEmb.shape)

    np.save('./'+str(MuType)+'.npy',saveEmb)





Collecting: 100%|██████████| 16286/16286 [00:54<00:00, 301.06it/s]


(16286, 4096)


Collecting: 100%|██████████| 8544/8544 [00:55<00:00, 154.69it/s]


(8544, 4096)
SenWordFuzz


100%|██████████| 1417/1417 [01:28<00:00, 15.96it/s]


(1417, 3, 4096)
swapGender_sentiSynonymSubstitution


100%|██████████| 681/681 [00:48<00:00, 14.07it/s]


(681, 3, 4096)
synon_contr


100%|██████████| 588/588 [01:08<00:00,  8.63it/s]

(588, 3, 4096)
